In [1]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
root_dir   = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(root_dir,'code'))
from model_src import fwrf_fit as fwrf_fit
from model_src import fwrf_predict as fwrf_predict
from model_src import texture_statistics_gabor, texture_statistics_pyramid

from model_fitting import initialize_fitting

fpX = np.float32

In [2]:
import torch
out = torch.load('/user_data/mmhender/imStat/model_fits/S01/texture_pyramid_ridge_4ori_4sf/Aug-15-2021_2220_DEBUG/all_fit_params')

In [3]:
out.keys()

dict_keys(['aperture', 'aperture_rf_range', 'models', 'feature_info', 'voxel_mask', 'brain_nii_shape', 'image_order', 'voxel_index', 'voxel_roi', 'voxel_ncsnr', 'best_params', 'lambdas', 'best_lambdas', 'best_losses', 'val_cc', 'val_r2', 'zscore_features', 'nonlin_fn', 'n_prf_sd_out', 'debug', 'up_to_sess', 'shuff_rnd_seed'])

In [9]:
out['best_params'][1].shape

(14913, 641, 15)

In [31]:
out['val_cc'][100,:]

array([-0.06987316, -0.07483465, -0.0713683 , -0.04560008, -0.10405765,
       -0.0706954 , -0.06468303, -0.03063224, -0.09032618, -0.05314833,
       -0.09400647, -0.05596919,  0.00536413, -0.05441273, -0.06036349],
      dtype=float32)

In [2]:
import sys
import os
import struct
import time
import numpy as np
import h5py
from tqdm import tqdm
import pickle
import math
import sklearn
from sklearn import decomposition
import gc
import copy

import torch
import torch.nn as nn
import torch.nn.init as I
import torch.nn.functional as F
import torch.optim as optim

from utils import numpy_utility, torch_utils
from model_src import fwrf_fit as fwrf_fit
from model_src import texture_statistics_gabor as texture_statistics


In [3]:
import sys
import os
import struct
import time
import numpy as np
import h5py
from tqdm import tqdm
import pickle
import math
import sklearn
from sklearn import decomposition

import torch
import torch.nn as nn
import torch.nn.init as I
import torch.nn.functional as F
import torch.optim as optim

from utils import numpy_utility, torch_utils

In [2]:
root_dir

'/user_data/mmhender/imStat'

In [4]:
subject=1
roi=None

ridge=0

shuffle_images=0
random_images=0
random_voxel_data=0

sample_batch_size=100
voxel_batch_size=100
zscore_features=1
nonlin_fn=0
padding_mode='circular'

n_ori=4
n_sf=4
up_to_sess=1
debug=1
shuff_rnd_seed=0
# shuff_rnd_seed=251709

fitting_type='texture'

do_fitting=1
do_val=1
do_partial=1
date_str=None

In [5]:
device = initialize_fitting.init_cuda()
# device = torch.device('cpu:0')
nsd_root, stim_root, beta_root, mask_root = initialize_fitting.get_paths()
model_name, feature_types_exclude = initialize_fitting.get_pyramid_model_name(ridge, n_ori, n_sf)

if do_fitting==False and date_str is None:
    raise ValueError('if you want to start midway through the process (--do_fitting=False), then specify the date when training result was saved (--date_str).')

if do_fitting==True and date_str is not None:
    raise ValueError('if you want to do fitting from scratch (--do_fitting=True), specify --date_str=None (rather than entering a date)')

output_dir, fn2save = initialize_fitting.get_save_path(root_dir, subject, model_name, shuffle_images, random_images, random_voxel_data, debug, date_str)

#device: 1
device#: 0
device name: GeForce GTX 1080 Ti

torch: 1.8.1+cu111
cuda:  11.1
cudnn: 8005
dtype: torch.float32
Time Stamp: Aug-15-2021_1626

Will save final output file to /user_data/mmhender/model_fits/S01/texture_pyramid_OLS_4ori_4sf/Aug-15-2021_1626_DEBUG/



In [6]:
# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = initialize_fitting.get_voxel_info(mask_root, beta_root, subject, roi)

# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_single_trial_data, val_voxel_single_trial_data, \
    n_voxels, n_trials_val, image_order = initialize_fitting.get_data_splits(nsd_root, beta_root, stim_root, subject, voxel_mask, up_to_sess, 
                                                                             shuffle_images=shuffle_images, random_images=random_images, random_voxel_data=random_voxel_data)

# Need a multiple of 8
process_at_size=240
trn_stim_data = skimage.transform.resize(trn_stim_data, output_shape=(trn_stim_data.shape[0],1,process_at_size, process_at_size))
val_stim_single_trial_data = skimage.transform.resize(val_stim_single_trial_data, output_shape=(val_stim_single_trial_data.shape[0],1,process_at_size, process_at_size))

# Set up the pyramid
_fmaps_fn = texture_statistics_pyramid.steerable_pyramid_extractor(pyr_height=n_sf, n_ori = n_ori)
# Params for the spatial aspect of the model (possible pRFs)
#     aperture_rf_range=0.8 # using smaller range here because not sure what to do with RFs at edges...
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    

# Initialize the "texture" model which builds on first level feature maps
n_prf_sd_out=2
_texture_fn = texture_statistics_pyramid.texture_feature_extractor(_fmaps_fn,sample_batch_size=sample_batch_size, feature_types_exclude=feature_types_exclude, n_prf_sd_out=n_prf_sd_out, aperture=aperture, device=device)

# More params for fitting
holdout_size, lambdas = initialize_fitting.get_fitting_pars(trn_voxel_data, zscore_features, ridge=ridge)



3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[-1.  0. 26. 27. 28. 30. 31. 32. 33.]
1535 voxels are defined (differently) in both retinotopic areas and category areas

14913 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 'IPS3', 'IPS4', 'IPS5', 'SPL1', 'FEF']
[1, 2, 3, 4, 5]
['OFA', 'FFA-1', 'FFA-2', 'mTL-faces', 'aTL-faces']
[1, 2, 3]
['O

In [54]:
gc.collect()
torch.cuda.empty_cache()
print('\nStarting training...\n')
if shuff_rnd_seed==0:
    shuff_rnd_seed = int(time.strftime('%M%H%d', time.localtime()))
best_losses, best_lambdas, best_params, feature_info = fit_texture_model_ridge_varpart(trn_stim_data, trn_voxel_data, _texture_fn, models, lambdas, \
    zscore=zscore_features, voxel_batch_size=voxel_batch_size, holdout_size=holdout_size, shuffle=True, add_bias=True, debug=debug, 
                                                                                       shuff_rnd_seed=shuff_rnd_seed,device=device, do_varpart=False)




Starting training...

trn_size = 619 (90.0%)
dtype = <class 'numpy.float32'>
device = cuda:0
---------------------------------------
Seeding random number generator: seed is 271615

Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Computing higher order correlations...
time elapsed: 25.48157 s
time elapsed: 25.51060 s
time elapsed: 25.49836 s
time elapsed: 25.50260 s
time elapsed: 25.50631 s
time elapsed: 25.50986 s
time elapsed: 22.43819 s
time elapsed = 180.12879
Final size of features concatenated is [688 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']

Fitting 

In [57]:
best_models, weights, bias, features_mt, features_st, best_model_inds, partial_version_names = best_params
    

In [55]:
for pp in best_params:
    print(pp.shape)

(14913, 1, 3)
(14913, 641, 1)
(14913, 1)
(14913, 641)
(14913, 641)
(14913, 1)


AttributeError: 'list' object has no attribute 'shape'

In [61]:
# weights[0,:]

In [30]:

def fit_texture_model_ridge_varpart(images, voxel_data, _texture_fn, models, lambdas, zscore=False, voxel_batch_size=100, 
                            holdout_size=100, shuffle=True, add_bias=False, debug=False, shuff_rnd_seed=0, device=None, do_varpart=True):
   
    """
    Solve for encoding model weights using ridge regression.
    Inputs:
        images: the training images, [n_trials x 1 x height x width]
        voxel_data: the training voxel data, [n_trials x n_voxels]
        _texture_fn: module that maps from images to texture model features
        models: the list of possible pRFs to test, columns are [x, y, sigma]
        lambdas: ridge lambda parameters to test
        zscore: want to zscore each column of feature matrix before fitting?
        voxel_batch_size: how many voxels to use at a time for model fitting
        holdout_size: how many training trials to hold out for computing loss/lambda selection?
        shuffle: do we shuffle training data order before holding trials out?
        add_bias: add a column of ones to feature matrix, for an additive bias?
        debug: want to run a shortened version of this, to test it?
        shuff_rnd_seed: if we do shuffle training data (shuffle=True), what random seed to use? if zero, choose a new random seed in this code.
    Outputs:
        best_losses: loss value for each voxel (with best pRF and best lambda), eval on held out set
        best_lambdas: best lambda for each voxel (chosen based on loss w held out set)
        best_params: 
            [0] best pRF for each voxel [x,y,sigma]
            [1] best weights for each voxel/feature
            [2] if add_bias=True, best bias value for each voxel
            [3] if zscore=True, the mean of each feature before z-score
            [4] if zscore=True, the std of each feature before z-score
            [5] index of the best pRF for each voxel (i.e. index of row in "models")
        feature_info: describes types of features in texture model, see texture_feature_extractor in texture_statistics.py
        
    """
   
    dtype = images.dtype.type
    if device is None:
        device=torch.device('cpu:0')
#     device = next(_texture_fn.parameters()).device
    trn_size = len(voxel_data) - holdout_size
    assert trn_size>0, 'Training size needs to be greater than zero'
    
    print ('trn_size = %d (%.1f%%)' % (trn_size, float(trn_size)*100/len(voxel_data)))
    print ('dtype = %s' % dtype)
    print ('device = %s' % device)
    print ('---------------------------------------')
    
    # First do shuffling of data and define set to hold out
    n_trials = len(images)
    n_prfs = len(models)
    n_voxels = voxel_data.shape[1]
    order = np.arange(len(voxel_data), dtype=int)
    if shuffle:
        if shuff_rnd_seed==0:
            print('Computing a new random seed')
            shuff_rnd_seed = int(time.strftime('%M%H%d', time.localtime()))
        print('Seeding random number generator: seed is %d'%shuff_rnd_seed)
        np.random.seed(shuff_rnd_seed)
        np.random.shuffle(order)
    images = images[order]
    voxel_data = voxel_data[order]  
    trn_data = voxel_data[:trn_size]
    out_data = voxel_data[trn_size:]

    n_features_total = _texture_fn.n_features_total
    n_feature_types = len(_texture_fn.feature_types_include)
    if do_varpart:
        n_partial_versions = n_feature_types+1
        partial_version_names = ['full_model']+['leave_out_%s'%ff for ff in _texture_fn.feature_types_include]
        masks = np.concatenate([np.expand_dims(np.array(_texture_fn.feature_column_labels!=ff).astype('int'), axis=0) for ff in np.arange(-1,n_feature_types)], axis=0)
    else:
        n_partial_versions = 1;  
        partial_version_names = ['full_model']
        masks = np.ones([1,n_features_total])
    # "partial versions" will be listed as: [full model, leave out first set of features, leave out second set of features...]

    if add_bias:
        masks = np.concatenate([masks, np.ones([masks.shape[0],1])], axis=1) # always include intercept 
    masks = np.transpose(masks)
    # masks is [n_features_total (including intercept) x n_partial_versions]

    # Create full model value buffers    
    best_models = np.full(shape=(n_voxels,n_partial_versions), fill_value=-1, dtype=int)   
    best_lambdas = np.full(shape=(n_voxels,n_partial_versions), fill_value=-1, dtype=int)
    best_losses = np.full(fill_value=np.inf, shape=(n_voxels,n_partial_versions), dtype=dtype)
    # creating a third dim here, listing the "partial" versions of the model (setting to zero a subset of features at a time)
    best_w_params = np.zeros(shape=(n_voxels, n_features_total,n_partial_versions), dtype=dtype)

    if add_bias:
        best_w_params = np.concatenate([best_w_params, np.ones(shape=(n_voxels,1,n_partial_versions), dtype=dtype)], axis=1)

    features_mean = None
    features_std = None
    if zscore:
        features_mean = np.zeros(shape=(n_voxels, n_features_total), dtype=dtype)
        features_std  = np.zeros(shape=(n_voxels, n_features_total), dtype=dtype)
    
    
    start_time = time.time()
    vox_loop_time = 0
    print ('')
    
    with torch.no_grad():
        
        # Looping over models (here models are different spatial RF definitions)
        for m,(x,y,sigma) in enumerate(models):
            if debug and m>1:
                break
            print('\nGetting features for prf %d: [x,y,sigma] is [%.2f %.2f %.4f]'%(m, models[m,0],  models[m,1],  models[m,2] ))
            
            t = time.time()   
            
            # Get features for the desired pRF, across all trn set image   
        
            all_feat_concat, feature_info = _texture_fn(images, [x,y,sigma])
            
            features = torch_utils.get_value(all_feat_concat)
            
            elapsed = time.time() - t
        
            if zscore:  
                features_m = np.mean(features, axis=0, keepdims=True) #[:trn_size]
                features_s = np.std(features, axis=0, keepdims=True) + 1e-6          
                features -= features_m
                features /= features_s    
                
            if add_bias:
                features = np.concatenate([features, np.ones(shape=(len(features), 1), dtype=dtype)], axis=1)
            
            # separate design matrix into training/held out data (for lambda selection)
            trn_features = features[:trn_size]
            out_features = features[trn_size:]   

            zero_columns = np.sum(trn_features[:,0:-1], axis=0)==0
            if np.sum(zero_columns)>0:
                print('n zero columns: %d'%np.sum(zero_columns))
                for ff in range(len(feature_info[1])):
                    if np.sum(zero_columns[feature_info[0]==ff])>0:
                        print('   %d columns are %s'%(np.sum(zero_columns[feature_info[0]==ff]), feature_info[1][ff]))

            # Looping over versions of model w different features set to zero (variance partition)
            for pp in range(n_partial_versions):
                
                print('\nFitting version %d of %d: %s, '%(pp, n_partial_versions, partial_version_names[pp]))

                nonzero_inds = masks[:,pp]==1
                best_w_tmp = best_w_params[:,nonzero_inds,pp] # chunk of the full weights matrix to work with for this partial model

                # Send matrices to gpu
                _xtrn = torch_utils._to_torch(trn_features[:,nonzero_inds], device=device)
                _xout = torch_utils._to_torch(out_features[:,nonzero_inds], device=device)   

                # Do part of the matrix math involved in ridge regression optimization out of the loop, 
                # because this part will be same for all the voxels.
                _cof = _cofactor_fn_cpu(_xtrn, lambdas)

                # Now looping over batches of voxels (only reason is because can't store all in memory at same time)
                vox_start = time.time()
                for rv,lv in numpy_utility.iterate_range(0, n_voxels, voxel_batch_size):
                    sys.stdout.write('\rVoxels [%6d:%-6d] of %d' % (m, n_prfs, rv[0], rv[-1], n_voxels))

                    # Send matrices to gpu
                    _vtrn = torch_utils._to_torch(trn_data[:,rv], device=device)
                    _vout = torch_utils._to_torch(out_data[:,rv], device=device)

                    # Here is where optimization happens - relatively simple matrix math inside loss fn.
                    _betas, _loss = _loss_fn(_cof, _vtrn, _xout, _vout) #   [#lambda, #feature, #voxel, ], [#lambda, #voxel]
                    # Now have a set of weights (in betas) and a loss value for every voxel and every lambda. 
                    # goal is then to choose for each voxel, what is the best lambda and what weights went with that lambda.

                    # first choose best lambda value and the loss that went with it.
                    _values, _select = torch.min(_loss, dim=0)
                    betas = torch_utils.get_value(_betas)
                    values, select = torch_utils.get_value(_values), torch_utils.get_value(_select)

                    # comparing this loss to the other models for each voxel (e.g. the other RF position/sizes)
                    imp = values<best_losses[rv,pp]

                    if np.sum(imp)>0:                    
                        # for whichever voxels had improvement relative to previous models, save parameters now
                        # this means we won't have to save all params for all models, just best.
                        arv = np.array(rv)[imp]
                        li = select[imp]
                        best_lambdas[arv,pp] = li
                        best_losses[arv,pp] = values[imp]
                        best_models[arv,pp] = m
                        if zscore:
                            features_mean[arv] = features_m # broadcast over updated voxels
                            features_std[arv]  = features_s
                        # taking the weights associated with the best lambda value
                        best_w_tmp[arv,:] = numpy_utility.select_along_axis(betas[:,:,imp], li, run_axis=2, choice_axis=0).T

                best_w_params[:,nonzero_inds,pp] = best_w_tmp

                vox_loop_time += (time.time() - vox_start)
                elapsed = (time.time() - vox_start)
                sys.stdout.flush()

    # Print information about how fitting went...
    total_time = time.time() - start_time
    inv_time = total_time - vox_loop_time
    return_params = [best_w_params[:,:n_features_total,:]]
    if add_bias:
        return_params += [best_w_params[:,-1,:]]
    else: 
        return_params += [None,]
    print ('\n---------------------------------------')
    print ('total time = %fs' % total_time)
    print ('total throughput = %fs/voxel' % (total_time / n_voxels))
    print ('voxel throughput = %fs/voxel' % (vox_loop_time / n_voxels))
    print ('setup throughput = %fs/model' % (inv_time / n_prfs))
    sys.stdout.flush()
    
    best_params = [models[best_models],]+return_params+[features_mean, features_std]+[best_models]+[partial_version_names]
    
    return best_losses, best_lambdas, best_params, feature_info


In [24]:

def _cofactor_fn_cpu(_x, lambdas):
    '''
    Generating a matrix needed to solve ridge regression model for each lambda value.
    Ridge regression (Tikhonov) solution is :
    w = (X^T*X + I*lambda)^-1 * X^T * Y
    This func will return (X^T*X + I*lambda)^-1 * X^T. 
    So once we have that, can just multiply by training data (Y) to get weights.
    returned size is [nLambdas x nFeatures x nTrials]
    This version makes sure that the torch inverse operation is done on the cpu, and in floating point-64 precision.
    Otherwise get bad results for small lambda values. This seems to be a torch-specific bug.
    
    '''
    device_orig = _x.device
    type_orig = _x.dtype
    # switch to this specific format which works with inverse
    _x = _x.to('cpu').to(torch.float64)
    _f = torch.stack([(torch.mm(torch.t(_x), _x) + torch.eye(_x.size()[1], device='cpu', dtype=torch.float64) * l).inverse() for l in lambdas], axis=0) 
    
    # [#lambdas, #feature, #feature] 
    cof = torch.tensordot(_f, _x, dims=[[2],[1]]) # [#lambdas, #feature, #sample]
    
    # put back to whatever way it was before, so that we can continue with other operations as usual
    return cof.to(device_orig).to(type_orig)



def _loss_fn(_cofactor, _vtrn, _xout, _vout):
    '''
    Calculate loss given "cofactor" from cofactor_fn, training data, held-out design matrix, held out data.
    returns weights (betas) based on equation
    w = (X^T*X + I*lambda)^-1 * X^T * Y
    also returns loss for these weights w the held out data. SSE is loss func here.
    '''

    _beta = torch.tensordot(_cofactor, _vtrn, dims=[[2], [0]]) # [#lambdas, #feature, #voxel]
    _pred = torch.tensordot(_xout, _beta, dims=[[1],[1]]) # [#samples, #lambdas, #voxels]
    _loss = torch.sum(torch.pow(_vout[:,None,:] - _pred, 2), dim=0) # [#lambdas, #voxels]
    return _beta, _loss


In [41]:
best_models, weights, bias, features_mt, features_st, best_model_inds = best_params
    

In [52]:
weights.shape

(14913, 641, 15)

In [61]:
val_cc_partial[0,:]

array([ 0.08345806, -0.10948003, -0.12350307,  0.1346478 ,  0.06219026,
        0.06962853,  0.12246101, -0.1408787 , -0.01380709, -0.05563707,
       -0.21057074,  0.02978912,  0.16556154, -0.00244683, -0.09867395],
      dtype=float32)

In [63]:
gc.collect()
torch.cuda.empty_cache()
# val_cc, val_r2 = fwrf_predict.validate_texture_model(best_params, models, val_voxel_single_trial_data, val_stim_single_trial_data, \
#                                                      _texture_fn, sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, \
#                                                      debug=debug, dtype=fpX)

val_cc_partial, val_r2_partial = validate_texture_model_varpart(best_params, models, val_voxel_single_trial_data, \
                                                                             val_stim_single_trial_data, _texture_fn, \
                                                                             sample_batch_size=sample_batch_size, voxel_batch_size=voxel_batch_size, \
                                                                             debug=debug, dtype=fpX)




Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Computing higher order correlations...
time elapsed: 15.80200 s
time elapsed = 16.32994
Final size of features concatenated is [62 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']
Getting features for prf 1: [x,y,sigma] is [-0.49 -0.55 0.0400]
Computing higher order correlations...
time elapsed: 15.78480 s
time elapsed = 16.26832
Final size of features concatenated is [62 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', '

In [65]:
val_cc_partial.shape

(14913, 1)

In [62]:

def validate_texture_model_varpart(best_params, prf_models, val_voxel_single_trial_data, val_stim_single_trial_data, _texture_fn, sample_batch_size=100, voxel_batch_size=100, debug=False, dtype=np.float32):
    
    """ 
    Evaluate trained model, leaving out a subset of features at a time.
    """
    images = val_stim_single_trial_data
    params = best_params
    dtype = images.dtype.type
    device = _texture_fn.device
    
    n_trials, n_voxels = len(images), len(params[0])
    n_prfs = prf_models.shape[0]
    n_features = params[1].shape[1]  

    best_models, weights, bias, features_mt, features_st, best_model_inds, partial_version_names = params
    
    # val_cc is the correlation coefficient bw real and predicted responses across trials, for each voxel.
    n_voxels = np.shape(val_voxel_single_trial_data)[1]
    n_features_total = _texture_fn.n_features_total
    n_feature_types = len(_texture_fn.feature_types_include)
    n_partial_versions = len(partial_version_names)
    if n_partial_versions>1:
        masks = np.concatenate([np.expand_dims(np.array(_texture_fn.feature_column_labels!=ff).astype('int'), axis=0) for ff in np.arange(-1,n_feature_types)], axis=0)
    else:
        masks = np.ones([1,n_features_total])
    # "partial versions" will be listed as: [full model, leave out first set of features, leave out second set of features...]

    masks = np.transpose(masks)

    val_cc  = np.zeros(shape=(n_voxels, n_partial_versions), dtype=dtype)
    val_r2 = np.zeros(shape=(n_voxels, n_partial_versions), dtype=dtype)

    pred_models = np.full(fill_value=0, shape=(n_trials, n_features, n_prfs), dtype=dtype)
    
    start_time = time.time()    
    with torch.no_grad():
        
        # First gather texture features for all pRFs.
        for mm in range(n_prfs):
            if mm>1 and debug:
                break
            print('Getting features for prf %d: [x,y,sigma] is [%.2f %.2f %.4f]'%(mm, prf_models[mm,0],  prf_models[mm,1],  prf_models[mm,2] ))
            all_feat_concat, feature_info = _texture_fn(images,prf_models[mm,:])
            
            pred_models[:,:,mm] = torch_utils.get_value(all_feat_concat)
        
    
        vv=-1
        ## Looping over voxels here in batches, will eventually go through all.
        for rv, lv in numpy_utility.iterate_range(0, n_voxels, voxel_batch_size):
            vv=vv+1
            print('Getting predictions for voxels [%d-%d] of %d'%(rv[0],rv[-1],n_voxels))

            if vv>1 and debug:
                break
            
            # Looping over versions of model w different features set to zero (variance partition)
            for pp in range(n_partial_versions):
                
                print('Evaluating version %d of %d: %s'%(pp, n_partial_versions, partial_version_names[pp]))
   
                # [trials x features x voxels]
                features_full = pred_models[:,:,best_model_inds[rv,pp]]
                           
                nonzero_inds = masks[:,pp]==1
                
                features = features_full[:,nonzero_inds,:]

                # making sure to gather only the columns for features included in this partial model
                _weights = torch_utils._to_torch(weights[rv,:,pp][:,nonzero_inds]) 
                
                _bias = torch_utils._to_torch(bias[rv,pp])

            
                if features_mt is not None:
                    _features_m = torch_utils._to_torch(features_mt[rv,:][:,nonzero_inds])
                if features_st is not None:
                    _features_s = torch_utils._to_torch(features_st[rv,:][:,nonzero_inds])
                
                pred_block = np.full(fill_value=0, shape=(n_trials, lv), dtype=dtype)
                
                
                # Now looping over validation set trials in batches
                for rt, lt in numpy_utility.iterate_range(0, n_trials, sample_batch_size):

                    _features = torch_utils._to_torch(features[rt,:,:]) # trials x features x voxels
                    if features_mt is not None:    
                        # features_m is [nvoxels x nfeatures] - need [trials x features x voxels]
                        _features = _features - torch.tile(torch.unsqueeze(_features_m, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])

                    if features_st is not None:
                        _features = _features/torch.tile(torch.unsqueeze(_features_s, dim=0), [_features.shape[0], 1, 1]).moveaxis([1],[2])
                        _features[torch.isnan(_features)] = 0.0 # this applies in the pca case when last few columns of features are missing

                    # features is [#samples, #features, #voxels] - swap dims to [#voxels, #samples, features]
                    _features = torch.transpose(torch.transpose(_features, 0, 2), 1, 2)
                    # weights is [#voxels, #features]
                    # _r will be [#voxels, #samples, 1] - then [#samples, #voxels]

                    _r = torch.squeeze(torch.bmm(_features, torch.unsqueeze(_weights, 2)), dim=2).t() 

                    if _bias is not None:
                        _r = _r + torch.tile(torch.unsqueeze(_bias, 0), [_r.shape[0],1])

                    pred_block[rt] = torch_utils.get_value(_r) 

                # Now for this batch of voxels and this partial version of the model, measure performance.
#                 print('\nEvaluating correlation coefficient on validation set...\n')
                for vi in range(lv):   
                    val_cc[rv[vi],pp] = np.corrcoef(val_voxel_single_trial_data[:,rv[vi]], pred_block[:,vi])[0,1]  
                    val_r2[rv[vi],pp] = get_r2(val_voxel_single_trial_data[:,rv[vi]], pred_block[:,vi])
                
                sys.stdout.flush()
        
    val_cc = np.nan_to_num(val_cc)
    val_r2 = np.nan_to_num(val_r2) 
    
    return val_cc, val_r2


In [29]:
   
def get_r2(actual,predicted):
  
    """
    This computes the coefficient of determination (R2).
    For OLS, this is a good measure of variance explained. 
    Not necessarily true for ridge regression - can use signed correlation coefficient^2 instead.
    With OLS & when train/test sets are identical, R2 = correlation coefficient^2.
    """
    
    # calculate r2 for this fit.
    ssres = np.sum(np.power((predicted - actual),2));
    sstot = np.sum(np.power((actual - np.mean(actual)),2));
    r2 = 1-(ssres/sstot)
    
    return r2
 
